<a href="https://colab.research.google.com/github/elizavetkachenko/portfolio/blob/main/python/A_B_testing_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing the data

[Link to csv file](https://drive.google.com/file/d/1Qd8E0CMf0CSfJO4o8j-pWpv041W93HzF/view?usp=sharing)

In [ ]:
import numpy as np
import pandas as pd

from google.colab import drive
drive.mount("/content/drive")
%cd /content/drive/MyDrive/Mate


df = pd.read_csv("ab_test_project.csv")
df.head()

Mounted at /content/drive
/content/drive/MyDrive/Mate


,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-02,Tunisia,desktop,Africa,Organic Search,2,1,new_account,1
1,2020-11-04,Jordan,desktop,Asia,Social Search,2,1,new_account,1
2,2020-11-05,New Zealand,desktop,Oceania,Undefined,2,1,new_account,1
3,2020-11-06,Serbia,desktop,Europe,Organic Search,2,1,new_account,1
4,2020-11-06,New Zealand,desktop,Oceania,Paid Search,2,1,new_account,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800996 entries, 0 to 800995
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   date        800996 non-null  object
 1   country     800996 non-null  object
 2   device      800996 non-null  object
 3   continent   800996 non-null  object
 4   channel     800996 non-null  object
 5   test        800996 non-null  int64 
 6   test_group  800996 non-null  int64 
 7   event_name  800996 non-null  object
 8   value       800996 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 55.0+ MB


In [ ]:
df['date'] = pd.to_datetime(df['date'])
df['test'] = df['test'].astype(str)
df['test_group'] = df['test_group'].astype(str)

In [ ]:
df['event_name'].value_counts()

,count
event_name,
session,107210
session_start,106242
page_view,101907
user_engagement,94520
first_visit,81621
scroll,73643
view_promotion,61695
view_item,44869
session_with_order,25892


# Calculations

In [ ]:
pivot_table = pd.pivot_table(
    df,
    values='value',
    index='event_name',
    columns=['test', 'test_group'],
    aggfunc='sum'
)
pivot_table

test                      1               2               3               4  \
test_group                1       2       1       2       1       2       1   
event_name                                                                    
add_payment_info       1988    2229    2344    2409    3623    3697    3731   
add_shipping_info      3034    3221    3480    3510    5298    5188    5128   
add_to_cart            1395    1366    2811    3061   17674   17229   21536   
begin_checkout         3784    4021    4262    4313    9532    9264   12555   
click                   368     353     337     413     280     292     285   
first_visit           30596   30512   34511   34171   50438   50728   80900   
new_account            3823    3681    4165    4184    5856    5822    8984   
page_view            191543  198050  220275  212320  286351  284287  379480   
scroll                73244   73376   80713   81370  110360  109564  136037   
select_item             543     530     905     946    8735    8556   12214   
select_promotion       1275    1323    1477    1406    2020    1996    2773   
session               45362   45193   50637   50244   70047   70439  105079   
session_start         45905   45649   51219   50808   71312   71686  107128   
session_with_order     4514    4526    5102    5003    6951    7151   10596   
user_engagement      171788  179081  198266  189931  249921  248325  273946   
view_item             62335   65337   72717   68700   93931   92841   98619   
view_item_list           27      24      24      29       9       9       5   
view_promotion        29188   29117   32367   31680   41169   40959   52672   
view_search_results    3678    3882    4282    4198    5764    5619    7582   

test                         
test_group                2  
event_name                   
add_payment_info       3601  
add_shipping_info      4956  
add_to_cart           21554  
begin_checkout        12267  
click                   299  
first_visit           80626  
new_account            8687  
page_view            378307  
scroll               134156  
select_item           11988  
select_promotion       2708  
session              105141  
session_start        106978  
session_with_order    10481  
user_engagement      273633  
view_item             98928  
view_item_list            6  
view_promotion        51985  
view_search_results    7203

In [ ]:
from statsmodels.stats.proportion import proportions_ztest

# List of target metrics
target_metrics = [
    'add_to_cart',            # Added: start of commercial interest
    'begin_checkout',         # Start of checkout
    'add_shipping_info',      # Adding shipping info
    'add_payment_info',       # Adding payment info
    'session_with_order',     # Added: successful purchase (final result)
    'new_account'             # Registration (separate but important conversion)
]

results_df = pd.DataFrame()

all_tests = pivot_table.columns.get_level_values('test').unique()

for test_name in all_tests:

    # 1. Denominator: number of sessions
    # Group 1 (Control)
    sessions_cr = pivot_table.loc['session', (test_name, '1')]
    # Group 2 (Test)
    sessions_rr = pivot_table.loc['session', (test_name, '2')]

    # 2. Loop through each target metric
    for metric in target_metrics:

        # Numerator: number of events (metric)
        # Group 1 (Control)
        numerator_cr = pivot_table.loc[metric, (test_name, '1')]
        # Group 2 (Test)
        numerator_rr = pivot_table.loc[metric, (test_name, '2')]

        # 3. Calculate conversions and change
        conversion_cr = numerator_cr / sessions_cr
        conversion_rr = numerator_rr / sessions_rr

        # 4. Z-test
        metric_events = np.array([numerator_rr, numerator_cr])
        total_sessions = np.array([sessions_rr, sessions_cr])

        z_stat, p_value = proportions_ztest(metric_events, total_sessions, alternative='two-sided')

        # 5. Save the result as a new row in DataFrame
        new_row = {
            'test_number': test_name,
            'metric': f'{metric} / session',
            'numerator_cr': numerator_cr,
            'denominator_cr': sessions_cr,
            'conversion_cr': conversion_cr,
            'numerator_rr': numerator_rr,
            'denominator_rr': sessions_rr,
            'conversion_rr': conversion_rr,
            'metric_chang_z_stat': z_stat,
            'p_value': p_value,
            'significant': p_value < 0.05
        }

        results_df = pd.concat([results_df, pd.Series(new_row).to_frame().T], ignore_index=True)

results_df.head(24)


,test_number,metric,numerator_cr,denominator_cr,conversion_cr,numerator_rr,denominator_rr,conversion_rr,metric_chang_z_stat,p_value,significant
0,1,add_to_cart / session,1395,45362,0.030753,1366,45193,0.030226,-0.460924,0.644853,False
1,1,begin_checkout / session,3784,45362,0.083418,4021,45193,0.088974,2.978783,0.002894,True
2,1,add_shipping_info / session,3034,45362,0.066884,3221,45193,0.071272,2.603571,0.009226,True
3,1,add_payment_info / session,1988,45362,0.043825,2229,45193,0.049322,3.924884,0.000087,True
4,1,session_with_order / session,4514,45362,0.099511,4526,45193,0.100148,0.320049,0.748931,False
5,1,new_account / session,3823,45362,0.084278,3681,45193,0.081451,-1.542883,0.122859,False
6,2,add_to_cart / session,2811,50637,0.055513,3061,50244,0.060923,3.669417,0.000243,True
7,2,begin_checkout / session,4262,50637,0.084168,4313,50244,0.085841,0.952898,0.340642,False
8,2,add_shipping_info / session,3480,50637,0.068724,3510,50244,0.069859,0.709557,0.477979,False
9,2,add_payment_info / session,2344,50637,0.04629,2409,50244,0.047946,1.240994,0.214608,False


In [ ]:
results_df.to_csv("ab_test_results.csv", index=False)

from google.colab import files

files.download("ab_test_results.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Dashboard in Tableau Public

[Dashboard in Tableau Public](https://public.tableau.com/app/profile/yelyzaveta.tkachenko/viz/ABTestResultsAnalysis/ABTesting?publish=yes)

# Conclusion

**Test 1** was partly successful, showing a significant improvement in the middle part of the funnel (from begin_checkout to add_payment_info).
However, this success did not lead to a statistically significant increase in the final purchase stage (session_with_order).

**Test 2** showed a statistically significant improvement only at the first stage of the funnel (add_to_cart), but lost this effect in all following key steps, including the final purchase.
Therefore, the test should be continued until statistically significant results are achieved for other metrics.

**Test 3** should be considered a failure (Fail). The change caused a statistically significant decrease in conversion at the early stages of the funnel (from add_to_cart to begin_checkout).
Although the drop was not significant at the final stage (session_with_order), the negative impact is clear.

**Test 4** is also a failure (Fail). While the effect is less strong than in Test 3, the change led to a statistically significant drop in begin_checkout/session and new_account/session conversions.
This means the change worsened both the main commercial flow and the additional conversion (registration).